In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import GRU, LSTM, Dense, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from scipy.spatial.distance import cdist
import numpy as np

In [2]:
num_words = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [4]:
max_len = 256
embedding_size = 10
batch_size = 128
n_epochs = 10
pad_type = 'pre'

In [5]:
X_train_pad = pad_sequences(X_train,
                            maxlen=max_len,
                            padding=pad_type,
                            truncating=pad_type)
X_test_pad = pad_sequences(X_test,
                            maxlen=max_len,
                            padding=pad_type,
                            truncating=pad_type)

In [5]:
model = Sequential()

In [6]:
model.add(Embedding(input_dim=num_words,
                   output_dim=embedding_size,
                   input_length=max_len,
                   name='layer_embedding',))

In [7]:
model.add(Dropout(0.3))

In [8]:
model.add(LSTM(128, return_sequences=False))

In [9]:
model.add(Dropout(0.3))

In [10]:
model.add(Dense(1, activation='sigmoid', name='classification'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 10)           200000    
_________________________________________________________________
dropout (Dropout)            (None, 256, 10)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               71168     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
classification (Dense)       (None, 1)                 129       
Total params: 271,297
Trainable params: 271,297
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [13]:
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [14]:
%%time
model.fit(X_train_pad, y_train,
          epochs=n_epochs,
          batch_size=batch_size, 
          validation_split=0.05,
          callbacks=[callback_early_stopping]
         )

Epoch 1/10
186/186 [==============================] - 82s 440ms/step - loss: 0.5752 - acc: 0.6892 - val_loss: 0.4830 - val_acc: 0.7912
Epoch 2/10
186/186 [==============================] - 78s 420ms/step - loss: 0.3239 - acc: 0.8656 - val_loss: 0.3344 - val_acc: 0.8576
Epoch 3/10
186/186 [==============================] - 80s 432ms/step - loss: 0.2388 - acc: 0.9089 - val_loss: 0.3412 - val_acc: 0.8712
Epoch 4/10
186/186 [==============================] - 77s 412ms/step - loss: 0.1824 - acc: 0.9331 - val_loss: 0.3437 - val_acc: 0.8592
Epoch 5/10
186/186 [==============================] - 80s 430ms/step - loss: 0.1489 - acc: 0.9481 - val_loss: 0.3486 - val_acc: 0.8600
Epoch 6/10
186/186 [==============================] - 84s 449ms/step - loss: 0.1228 - acc: 0.9582 - val_loss: 0.3603 - val_acc: 0.8584
Epoch 7/10
186/186 [==============================] - 83s 448ms/step - loss: 0.1106 - acc: 0.9613 - val_loss: 0.4107 - val_acc: 0.8696
Epoch 00007: early stopping
Wall time: 9min 28s


In [15]:
%%time
eval_model = model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 40s 52ms/step - loss: 0.4291 - acc: 0.8589
Wall time: 40.4 s


In [16]:
print("Loss: {0:.5}".format(eval_model[0]))
print("Accuracy: {0:.2%}".format(eval_model[1]))

Loss: 0.42911
Accuracy: 85.89%


In [17]:
#gru model
model_GRU = Sequential()

In [18]:
model_GRU.add(Embedding(input_dim=num_words,
                       output_dim=embedding_size,
                       input_length=max_len,
                       name='layer_embedding'))

In [19]:
model_GRU.add(GRU(units=16, return_sequences=True))

In [20]:
model_GRU.add(GRU(units=8, return_sequences=True))

In [21]:
model_GRU.add(GRU(units=4, return_sequences=False))

In [22]:
model_GRU.add(Dense(1, activation='sigmoid'))

In [23]:
model_GRU.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 10)           200000    
_________________________________________________________________
gru (GRU)                    (None, 256, 16)           1344      
_________________________________________________________________
gru_1 (GRU)                  (None, 256, 8)            624       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 202,141
Trainable params: 202,141
Non-trainable params: 0
_________________________________________________________________


In [24]:
model_GRU.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [25]:
%%time
model_GRU.fit(X_train_pad, y_train, validation_split=0.05, epochs=n_epochs, batch_size=batch_size)

Epoch 1/10
186/186 [==============================] - 36s 195ms/step - loss: 0.5398 - accuracy: 0.7138 - val_loss: 0.6166 - val_accuracy: 0.7256
Epoch 2/10
186/186 [==============================] - 33s 180ms/step - loss: 0.3186 - accuracy: 0.8677 - val_loss: 0.3089 - val_accuracy: 0.8760
Epoch 3/10
186/186 [==============================] - 34s 182ms/step - loss: 0.2445 - accuracy: 0.9055 - val_loss: 0.4396 - val_accuracy: 0.8344
Epoch 4/10
186/186 [==============================] - 33s 180ms/step - loss: 0.2035 - accuracy: 0.9243 - val_loss: 0.3030 - val_accuracy: 0.8840
Epoch 5/10
186/186 [==============================] - 34s 182ms/step - loss: 0.1690 - accuracy: 0.9394 - val_loss: 0.4618 - val_accuracy: 0.8424
Epoch 6/10
186/186 [==============================] - 34s 183ms/step - loss: 0.1511 - accuracy: 0.9467 - val_loss: 0.4156 - val_accuracy: 0.8504
Epoch 7/10
186/186 [==============================] - 35s 187ms/step - loss: 0.1278 - accuracy: 0.9557 - val_loss: 0.3596 - val_ac

In [26]:
%%time
eval_GRU = model_GRU.evaluate(X_test_pad, y_test)

782/782 [==============================] - 22s 28ms/step - loss: 0.4162 - accuracy: 0.8624
Wall time: 21.9 s


In [27]:
print("Loss: {0:.5}".format(eval_GRU[0]))
print("Accuracy: {0:.2%}".format(eval_GRU[1]))

Loss: 0.41615
Accuracy: 86.24%


In [34]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def print_similar_words(word, metric='cosine'):
    
    token = word_index.get(word)

    embedding = weights_embedding[token]

    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    sorted_index = np.argsort(distances)
    
    sorted_distances = distances[sorted_index]
    
    sorted_words = [reverse_word_index[token] for token in sorted_index
                    if token != 0]

    def print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    N = 10

    print("Distance from '{0}':".format(word))

    print_words(sorted_words[0:N], sorted_distances[0:N])

    print("-------")

    print_words(sorted_words[-N:], sorted_distances[-N:])

In [30]:
layer_embedding = model.get_layer('layer_embedding')
weights_embedding = layer_embedding.get_weights()[0]

In [35]:
print_similar_words('good', metric='cosine')

Distance from 'good':
0.000 - good
0.073 - horrendously
0.093 - mobster
0.129 - sex
0.135 - within
0.153 - energy
0.155 - makers
0.157 - coming
0.170 - sticky
0.173 - payed
-------
1.838 - importantly
1.846 - afternoon
1.848 - whim
1.848 - andrew
1.849 - via
1.852 - thinks
1.853 - settling
1.885 - savage
1.888 - scenes
1.903 - threats


In [7]:
model_Conv_one = Sequential()

In [8]:
model_Conv_one.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [10]:
model_Conv_one.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [11]:
model_Conv_one.add(MaxPooling1D(pool_size=2))

In [12]:
model_Conv_one.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [13]:
model_Conv_one.add(MaxPooling1D(pool_size=2))

In [14]:
model_Conv_one.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [15]:
model_Conv_one.add(MaxPooling1D(pool_size=2))

In [17]:
model_Conv_one.add(Dropout(0.5))

In [20]:
model_Conv_one.add(Flatten())

In [22]:
model_Conv_one.add(Dense(250, activation='relu'))

In [23]:
model_Conv_one.add(Dense(1, activation='sigmoid', name='classification'))

In [25]:
model_Conv_one.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
model_Conv_one.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 10)           200000    
_________________________________________________________________
conv1d (Conv1D)              (None, 256, 128)          3968      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 128)          49280     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 128)          

In [28]:
%%time
model_Conv_one.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

Epoch 1/10
186/186 [==============================] - 36s 196ms/step - loss: 0.5485 - accuracy: 0.6910 - val_loss: 0.3232 - val_accuracy: 0.8704
Epoch 2/10
186/186 [==============================] - 39s 212ms/step - loss: 0.2932 - accuracy: 0.8813 - val_loss: 0.2845 - val_accuracy: 0.8880
Epoch 3/10
186/186 [==============================] - 37s 198ms/step - loss: 0.2153 - accuracy: 0.9164 - val_loss: 0.2916 - val_accuracy: 0.8840
Epoch 4/10
186/186 [==============================] - 34s 180ms/step - loss: 0.1732 - accuracy: 0.9347 - val_loss: 0.3145 - val_accuracy: 0.8912
Epoch 5/10
186/186 [==============================] - 34s 181ms/step - loss: 0.1383 - accuracy: 0.9498 - val_loss: 0.3123 - val_accuracy: 0.8856
Epoch 6/10
186/186 [==============================] - 34s 180ms/step - loss: 0.1104 - accuracy: 0.9603 - val_loss: 0.3559 - val_accuracy: 0.8896
Epoch 7/10
186/186 [==============================] - 34s 183ms/step - loss: 0.0911 - accuracy: 0.9673 - val_loss: 0.4031 - val_ac

In [29]:
%%time
eval_Conv_one = model_Conv_one.evaluate(X_test_pad, y_test)

782/782 [==============================] - 11s 14ms/step - loss: 0.5192 - accuracy: 0.8599
Wall time: 10.8 s


In [30]:
print("Loss: {0:.5}".format(eval_Conv_one[0]))
print("Accuracy: {0:.2%}".format(eval_Conv_one[1]))

Loss: 0.5192
Accuracy: 85.99%


In [31]:
model_Conv_two = Sequential()

In [33]:
model_Conv_two.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [34]:
model_Conv_two.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [36]:
model_Conv_two.add(MaxPooling1D(pool_size=3))

In [37]:
model_Conv_two.add(LSTM(128))

In [38]:
model_Conv_two.add(Dropout(0.3))

In [39]:
model_Conv_two.add(Dense(1, activation='sigmoid', name='classification'))

In [41]:
model_Conv_two.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
model_Conv_two.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

Epoch 1/10
186/186 [==============================] - 47s 255ms/step - loss: 0.4984 - accuracy: 0.7520 - val_loss: 0.3343 - val_accuracy: 0.8648
Epoch 2/10
186/186 [==============================] - 45s 243ms/step - loss: 0.2735 - accuracy: 0.8923 - val_loss: 0.3063 - val_accuracy: 0.8752
Epoch 3/10
186/186 [==============================] - 42s 227ms/step - loss: 0.2088 - accuracy: 0.9208 - val_loss: 0.3128 - val_accuracy: 0.8896
Epoch 4/10
186/186 [==============================] - 44s 237ms/step - loss: 0.1710 - accuracy: 0.9377 - val_loss: 0.3855 - val_accuracy: 0.8528
Epoch 5/10
186/186 [==============================] - 44s 236ms/step - loss: 0.1452 - accuracy: 0.9473 - val_loss: 0.3126 - val_accuracy: 0.8784
Epoch 6/10
186/186 [==============================] - 48s 258ms/step - loss: 0.1228 - accuracy: 0.9578 - val_loss: 0.3494 - val_accuracy: 0.8776
Epoch 7/10
186/186 [==============================] - 47s 250ms/step - loss: 0.1059 - accuracy: 0.9634 - val_loss: 0.3290 - val_ac

In [43]:
eval_Conv_two = model_Conv_two.evaluate(X_test_pad, y_test)

782/782 [==============================] - 20s 26ms/step - loss: 0.4359 - accuracy: 0.8597


In [44]:
print("Loss: {0:.5}".format(eval_Conv_two[0]))
print("Accuracy: {0:.2%}".format(eval_Conv_two[1]))

Loss: 0.43592
Accuracy: 85.97%
